In [1]:
from torch.utils.tensorboard import SummaryWriter
import os
import surpport.mySQL as mySQL
import surpport.myfunction as MF
from surpport.Args import Args
from surpport.dataprocess import dataload
import torch.nn.functional as F
import torch
import numpy as np

Using backend: pytorch


In [3]:
# mode rec
# from Experiment.Basline.NoneGraph.EEGnet import EEGNet
import sys
from importlib import reload
# sys.path.append(r'E:\CODEBASE\myDGL\FirstDGL\Experiment\Baseline\NoneGraph')
# import EEGNet
# reload(EEGNet)

sys.path.append(r'E:\CODEBASE\myDGL\FirstDGL\Experiment\Baseline\GraphModel\GNN')
import GNN
reload(GNN)

<module 'GNN' from 'E:\\CODEBASE\\myDGL\\FirstDGL\\Experiment\\Baseline\\GraphModel\\GNN\\GNN.py'>

In [4]:
# from surpport.nnstructure import Classifier7
def m_pre():
    arg = Args()
    # model = EEGNet.EEGNet2()
    # model = Classifier7(arg)
    model = GNN.GCN()
    model = model.cuda()
    return arg, model

def recorder_build(arg):
    base_rcd = mySQL.gen_base_rcd(arg)
    recorder = {'base': base_rcd}
    return recorder

In [6]:
names = ['kly']
fs = [2, ]
for name in names:
    for f in fs:
        arg, model = m_pre()
        arg.d_prepare(name, f)
        # ! 每次运行都改
        arg.m_info(m_name='eegnet2', m_task='220807_exp', num=5,)
        base_rcd = mySQL.gen_base_rcd(arg)
        recorder = {'base': base_rcd}
        writer = SummaryWriter(arg.tar_path+'\\Journal')

        # ! 根据模型修改
        opt_arg = {'params': model.parameters(),
                   'lr': 6e-5, 'eps': 1e-8, 'weight_decay': 0.1}
        opt = torch.optim.Adam(**opt_arg)

        arg, tr_dataloader, ts_dataloader = dataload(arg, model, opt)

In [7]:
for epoch in range(1, arg.epoch_num+1):
    print(epoch)
    recorder[str(epoch)+'-th'] = dict()
    rd = recorder[str(epoch)+'-th']
    # 由于图经过拼合，所以需要多一个dataloader的过程
    # 前两个是list
    tr_args = [epoch, model, opt, tr_dataloader, arg, writer]
    tr_loss, tr_acc = MF.train(*tr_args)
    mySQL.rcd_log(tr_loss, tr_acc, writer, rd, epoch, 'train')

    tr_args[3] = ts_dataloader
    el_loss, el_acc, logits, labels = MF.evaluate(*tr_args)

    mySQL.rcd_log(el_loss, el_acc, writer, rd, epoch, 'test')
    mySQL.rcd_result(logits, labels, rd)

    val_acc = np.mean(el_acc, axis=0)
    MF.save_best(val_acc, model, arg)

    mySQL.save_final(epoch, model, val_acc, arg, opt)

mySQL.save_recorder(recorder, arg, 'flow')
print('best acc: %.4f' % (arg.best_acc))
writer.close()

1
Epoch 01, Iter 005,train loss = 1.6042, train acc = 0.2812
Epoch 01, Iter 010,train loss = 1.6367, train acc = 0.1562
Epoch 01, Iter 015,train loss = 1.6494, train acc = 0.2500
Epoch 01, Iter 020,train loss = 1.6800, train acc = 0.1250
Epoch 01, Iter 025,train loss = 1.6773, train acc = 0.2188
Epoch 01, Iter 030,train loss = 1.6205, train acc = 0.1875
Epoch 01, Iter 035,train loss = 1.6484, train acc = 0.2188
Epoch 01, Iter 040,train loss = 1.6184, train acc = 0.2500
Epoch 01, Iter 045,train loss = 1.6832, train acc = 0.2188
Epoch 01, val loss = 1.6394, val acc = 0.2150
未达到期望，未保存模型
2
Epoch 02, Iter 005,train loss = 1.6352, train acc = 0.1562
Epoch 02, Iter 010,train loss = 1.6758, train acc = 0.2188
Epoch 02, Iter 015,train loss = 1.6106, train acc = 0.1562
Epoch 02, Iter 020,train loss = 1.6662, train acc = 0.1562
Epoch 02, Iter 025,train loss = 1.6081, train acc = 0.2500
Epoch 02, Iter 030,train loss = 1.6483, train acc = 0.1875
Epoch 02, Iter 035,train loss = 1.6199, train acc = 0